<a href="https://colab.research.google.com/github/ML-Bioinfo-CEITEC/ECCB2022/blob/main/notebooks/04a_Gradio_promoter_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -qq gradio transformers datasets

## Gradio Promoters

### Model Training

In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification

DATASET_NAME = "simecek/human_nontata_promoters"
dataset_train = load_dataset(DATASET_NAME, split='train')
dataset_test = load_dataset(DATASET_NAME, split='test[:1000]+test[-1000:]')

tokenizer = AutoTokenizer.from_pretrained("armheb/DNA_bert_6")
model = AutoModelForSequenceClassification.from_pretrained("simecek/DNADebertaK6b", num_labels=2)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

Downloading:   0%|          | 0.00/770 [00:00<?, ?B/s]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

...

Downloading:   0%|          | 0.00/28.7k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/704 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/187M [00:00<?, ?B/s]

Some weights of the model checkpoint at simecek/DNADebertaK6b were not used when initializing DebertaForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at simecek/DNA

In [7]:
def kmers(s, k=6):
  return [s[i:i + k] for i in range(0, len(s)-k+1)]

def tokenization(x): 
  return tokenizer(" ".join(kmers(x["seq"])), return_tensors="pt")

dataset_train_tokenized = dataset_train.map(tokenization, batched=False)
dataset_test_tokenized = dataset_test.map(tokenization, batched=False)

  0%|          | 0/27097 [00:00<?, ?ex/s]

  0%|          | 0/2000 [00:00<?, ?ex/s]

In [8]:
from transformers import TrainingArguments, Trainer

BATCH_SIZE = 32
LEARNING_RATE = 1e-5
EPOCHS = 3

training_arguments = TrainingArguments(
        output_dir='outputs', 
        learning_rate=LEARNING_RATE, 
        num_train_epochs=EPOCHS, 
        evaluation_strategy="epoch", 
        logging_strategy='epoch',
        per_device_train_batch_size=BATCH_SIZE, 
        per_device_eval_batch_size=BATCH_SIZE,
        fp16=True,
    )

trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=dataset_train_tokenized,
    eval_dataset=dataset_test_tokenized
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using cuda_amp half precision backend
The following columns in the training set don't have a corresponding argument in `DebertaForSequenceClassification.forward` and have been ignored: seq. If seq are not expected by `DebertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 27097
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 2541
/usr/local/lib/python3.7/dist-packages/transformers/models/deberta/modeling_deberta.py:679: UserWarning: To copy construct from a tensor, it is recommende

Epoch,Training Loss,Validation Loss
1,0.401000,0.319445
2,0.291200,0.324273
3,0.243400,0.273186


Saving model checkpoint to outputs/checkpoint-500
Configuration saved in outputs/checkpoint-500/config.json
Model weights saved in outputs/checkpoint-500/pytorch_model.bin
/usr/local/lib/python3.7/dist-packages/transformers/models/deberta/modeling_deberta.py:679: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  query_layer = query_layer / torch.tensor(scale, dtype=query_layer.dtype)
The following columns in the evaluation set don't have a corresponding argument in `DebertaForSequenceClassification.forward` and have been ignored: seq. If seq are not expected by `DebertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 32
Saving model checkpoint to outputs/checkpoint-1000
Configuration saved in outputs/checkpoint-1000/config.json
Model weights saved in

TrainOutput(global_step=2541, training_loss=0.3118719275848737, metrics={'train_runtime': 896.7347, 'train_samples_per_second': 90.652, 'train_steps_per_second': 2.834, 'total_flos': 5215389899620320.0, 'train_loss': 0.3118719275848737, 'epoch': 3.0})

### Pushing the model to HF Hub

In [9]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [10]:
model.push_to_hub("promoters_demo")

Configuration saved in /tmp/tmpm64xn_ii/config.json
Model weights saved in /tmp/tmpm64xn_ii/pytorch_model.bin
Uploading the following files to simecek/promoters_demo: config.json,pytorch_model.bin


### Gradio app

In [11]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("armheb/DNA_bert_6")
model2 = AutoModelForSequenceClassification.from_pretrained("simecek/promoters_demo")

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--armheb--DNA_bert_6/snapshots/a79a8fd96ad172f964a4dbef3f4d7545a5034baa/config.json
Model config BertConfig {
  "_name_or_path": "armheb/DNA_bert_6",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_ids": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_rnn_layer": 1,
  "output_past": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "rnn": "lstm",
  "rnn_dropout": 0.0,
  "rnn_hidden": 768,
  "split": 10,
  "transformers_version": "4.22.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 4101
}

loading file vocab.txt from cache at /ro

Downloading:   0%|          | 0.00/762 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--simecek--promoters_demo/snapshots/44652ac41596c8a48c5a59e5a6b6e90cd7542d81/config.json
Model config DebertaConfig {
  "_name_or_path": "simecek/promoters_demo",
  "architectures": [
    "DebertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 768,
  "pos_att_type": null,
  "position_biased_input": true,
  "relative_attention": false,
  "torch_dtype": "float32",
  "transformers_version": "4.22.1",
  "type_vocab_size": 0,
  "vocab_size": 4101
}



Downloading:   0%|          | 0.00/187M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--simecek--promoters_demo/snapshots/44652ac41596c8a48c5a59e5a6b6e90cd7542d81/pytorch_model.bin
All model checkpoint weights were used when initializing DebertaForSequenceClassification.

All the weights of DebertaForSequenceClassification were initialized from the model checkpoint at simecek/promoters_demo.
If your task is similar to the task the model of the checkpoint was trained on, you can already use DebertaForSequenceClassification for predictions without further training.


In [12]:
def kmers(s, k=6):
  return [s[i:i + k] for i in range(0, len(s)-k+1)]

def tokenization(x): 
  return tokenizer(" ".join(kmers(x["seq"])), return_tensors="pt")  ## note return_tensors="pt"

In [13]:
from scipy.special import softmax

seq = "AATGTTGAATCAATGAATGAGAGTGGGGCATAAAATGGGAAATTTGAGCACACAGAACTTTCATTTAACCTACTTCACTATTTGACTATGGTGTGGCCTTGCTTTTAATTGGAGGATAGCTGGAGTTGCAATCAGTATTCATGAAAGCATGTTTTTAAAAAAATGAGAATCAGTCTAACAGAATCTGAACTTTAAAGAACATCTCATTACTGATTATACATG"
seq = 'ATGGGTATCTGCCGAGATGGTAAAATGTCCAGTGGGCAGGAAATATTTCACCGGATCTCACTTGCCGGGGAAAAAAGCCAGTGAACGCGACCCACGCAGCTGACTAGATCGTGCGCTCGGCCTCAGATTGGTCGAGCCTGTTTGGGCTGCGGAAAAAGCGGAAGCGCGCTTTTCAAAGTCGTCCCACCATCAATGAAGGCAGGCCACTTCCGGCGTAGCCATGGCGGCTAACGCTACTACCAACCCGTCGC'

input = tokenization({"seq": seq})

logits = model2(**input)['logits'].detach().numpy()
probs = softmax(logits, axis=1)[0]

probs

/usr/local/lib/python3.7/dist-packages/transformers/models/deberta/modeling_deberta.py:679: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  query_layer = query_layer / torch.tensor(scale, dtype=query_layer.dtype)


array([0.01239971, 0.9876004 ], dtype=float32)

To create Gradio app, two things need to be specified:


*   Input(s) and output types (interface)
*   A function that will process input into output 



In [14]:
import gradio as gr

categories = ["not-promoter", "promoter"]

In [15]:
def is_promoter(DNAseq):
  input = tokenization({"seq": DNAseq})
  logits = model2(**input)['logits'].detach().numpy()
  probs = softmax(logits, axis=1)[0]
  probs = map(float, probs)

  return dict(zip(categories, probs))

In [16]:
is_promoter(seq)

/usr/local/lib/python3.7/dist-packages/transformers/models/deberta/modeling_deberta.py:679: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  query_layer = query_layer / torch.tensor(scale, dtype=query_layer.dtype)


{'not-promoter': 0.012399706989526749, 'promoter': 0.9876003861427307}

In [17]:
text = gr.inputs.Textbox(placeholder="Input DNA sequence", lines=5)
label = gr.outputs.Label(label = "Is it a promoter?")

intf = gr.Interface(fn=is_promoter, inputs=text, outputs=label)

intf.launch(debug=True)

/usr/local/lib/python3.7/dist-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  "Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components",
/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)
/usr/local/lib/python3.7/dist-packages/gradio/outputs.py:197: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://17634.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces


/usr/local/lib/python3.7/dist-packages/transformers/models/deberta/modeling_deberta.py:679: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  query_layer = query_layer / torch.tensor(scale, dtype=query_layer.dtype)


Keyboard interruption in main thread... closing server.


(<gradio.routes.App at 0x7f08eebf5190>,
 'http://127.0.0.1:7860/',
 'https://17634.gradio.app')

For hosting an app on HF Spaces, you need first to get yourself HF account, create an app (description) and then just [upload the files](https://huggingface.co/spaces/jph00/pets/tree/main) into HF Space repo. See [HF course](https://huggingface.co/course/chapter9/1) for details.